In [17]:
import torch
import numpy as np
import torch.nn as nn
from torchvision import transforms
import torchvision
import torch.nn.functional as F
import torch.optim as optim

In [3]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
bs=8

In [4]:
train=torchvision.datasets.CIFAR10(root="./Pytorch_Datasets",train=True,download=True,transform=transform)
train_loader=torch.utils.data.DataLoader(train,batch_size=bs,shuffle=True,num_workers=2)
test=torchvision.datasets.CIFAR10(root="./Pytorch_Datasets",train=False,download=False,transform=transform)
testloader = torch.utils.data.DataLoader(train, batch_size = bs, shuffle = False, num_workers = 2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [10]:
m=nn.Conv2d(in_channels=2,out_channels=6,kernel_size=5,stride=2,padding=2)
# input = torch.ones(1,3, 5,5)
# output = m(input)
m

Conv2d(2, 6, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))

In [8]:
m = nn.Conv2d(3,4, 2, stride=2)
# non-square kernels and unequal stride and with padding
#m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
# non-square kernels and unequal stride and with padding and dilation
#m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))
input = torch.ones(1,3, 5,5)
output = m(input)


In [9]:
output

tensor([[[[-0.1243, -0.1243],
          [-0.1243, -0.1243]],

         [[-0.3509, -0.3509],
          [-0.3509, -0.3509]],

         [[ 1.0723,  1.0723],
          [ 1.0723,  1.0723]],

         [[ 0.5944,  0.5944],
          [ 0.5944,  0.5944]]]], grad_fn=<ThnnConv2DBackward>)

In [14]:
train_loader,testloader,train,test

(<torch.utils.data.dataloader.DataLoader at 0x26f7804ac48>,
 <torchvision.datasets.cifar.CIFAR10 at 0x26f78071ac8>)

In [38]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5)
        self.max1=nn.MaxPool2d(2,2)
        self.relu1=nn.ReLU()
        self.conv2=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5)
        self.max2=nn.MaxPool2d(2,2)
        self.relu2=nn.ReLU()
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10) 
        
    def forward(self,x):
        out=self.conv1(x)
        out=self.max1(out)
        out=self.relu1(out)
        out=self.conv2(out)
        out=self.max2(out)
        out=self.relu2(out)
        
        out=out.view(-1, 16 *5*5)
        out=self.fc1(out)
        out=self.fc2(out)
        out=self.fc3(out)
        return out
    
net=Net()

In [25]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()\
        #The init section of the class creates the layers needed for the NN
        self.conv1 = nn.Conv2d(3,6,5) #in channels, out channels, kernel size
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10) #Final layer here is linear prediction of each categories. Typically we'd use a softmax instead
        
    def forward(self, x):
        #Create the foreward method to step through each of the layers created above
        #Note that the output of each layer is the input to the next layer
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 *5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [39]:
crit = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr= 0.0001, momentum = 0.9)

In [40]:
for epoch in range(4):
    running_loss=0.0
    for i,data in enumerate(train_loader,0):
        image,label=data
         #zero out the parameter gradients
        opt.zero_grad()
        #Automatically perform forward step
        output=net(image)
         #Calculate Loss Function
        loss=crit(output,label)
        #TAke the derivative of the loss function
        loss.backward()
         #opt.step takes the new gradients calculated in .backward multiplies by learning rate, then updates net parameters
        opt.step()
        running_loss += loss.item()
        
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch +1, i+1, running_loss/2000))
            running_loss = 0.0
print('Finished Training')

[1,  2000] loss: 2.303
[1,  4000] loss: 2.296
[1,  6000] loss: 2.273
[2,  2000] loss: 2.190
[2,  4000] loss: 2.031
[2,  6000] loss: 1.918
[3,  2000] loss: 1.831
[3,  4000] loss: 1.773
[3,  6000] loss: 1.717
[4,  2000] loss: 1.674
[4,  4000] loss: 1.649
[4,  6000] loss: 1.618
Finished Training
